<h1>Video Sliding Windows</h1>

<p>
So far we restricted ourselves to 1D time series, but the idea of recovering periodic dynamics with geometry can just as easily apply to multivariate signals.  In this module, we will examine sliding windows of videos as an exmaple.  Many natural videos also have periodicity, such as this video of a woman doing jumping jacks
</p>

<video controls>
  <source src="jumpingjacks.ogg" type="video/ogg">
Your browser does not support the video tag.
</video>

<p>
Video can be decomposed into a 3D array, which has dimensions width x height x time.  To tease out periodicity in geometric form, we will do the exact same thing as with sliding window 1D signal embeddings, but instead of just one sample per time shift, we need to take every pixel in every frame in the time window.  The figure below depicts this
</p>

<img src = "VideoStackTime.svg"><BR><BR>

To see this visually in the video next to PCA of the embedding, look at the following video

[Jumping jacks sliding window PCA video]

*Make a remark about practical issues with delay embeddings of videos; need PCA


In [5]:
#Do all of the imports and setup inline plotting
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import scipy.interpolate

from TDA import *
from VideoTools import *

In [ ]:
def getPCAVideo(I):
    ICov = I.dot(I.T)
    [lam, V] = linalg.eigh(ICov)
    V = V*np.sqrt(lam[None, :])
    return V

In [6]:
def getSlidingWindowVideo(I, dim, Tau, dT):
    N = I.shape[0] #Number of frames
    P = I.shape[1] #Number of pixels (possibly after PCA)
    pix = np.arange(P)
    NWindows = int(np.floor((N-dim*Tau)/dT))
    X = np.zeros((NWindows, dim*P))
    idx = np.arange(N)
    for i in range(NWindows):
        idxx = dT*i + Tau*np.arange(dim)
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))+2
        if end >= len(x):
            X = X[0:i, :]
            break
        f = scipy.interpolate.interp2d(pix, idx[start:end+1], I[idx[start:end+1], :], kind='linear')
        X[i, :] = f(pix, idxx).flatten()
    return X

In [4]:
#Load in video and do PCA to compress dimension
(X, FrameDims) = loadVideo("jumpingjacks.ogg")
print("Doing PCA for dimensionality reduction...")
X = getPCAVideo(X)
print("Finished PCA")

In [3]:
#Given that the period is 30 frames per cycle, choose a dimension and a Tau that capture 
#this motion in the roundest possible way
#Plot persistence diagram and PCA
dim = 20
Tau = 0.1
dT = 1

#Get sliding window video
XS = getSlidingWindowVideo(X, dim, Tau, dT)

#Mean-center and normalize sliding window
XS = XS - np.mean(XS, 1)[:, None]
XS = XS/np.sqrt(np.sum(XS**2, 1))[:, None]

#Get persistence diagrams
PDs = doRipsFiltration(XS, 1)

#Do PCA for visualization
pca = PCA(n_components = 3)
Y = pca.fit_transform(XS)


fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121)
plotDGMAx(ax1, PDs[1])
ax1.set_title("1D Persistence Diagram")

c = plt.get_cmap('jet')
C = c(np.array(np.round(np.linspace(0, 255, Y.shape[0])), dtype=np.int32))
C = C[:, 0:3]
ax2 = fig.add_subplot(122, projection = '3d')
ax2.set_title("PCA of Sliding Window Embedding")
ax2.scatter(Y[:, 0], Y[:, 1], Y[:, 2], c=C)
ax2.set_aspect('equal', 'datalim')
plt.show()

<h1>Summary</h1>
<ul>
<li>Periodicity can be studied on general time series data, including multivariate time series such as video</li>
</ul>